In [1]:
import pandas as pd 
import csv
import bioframe as bf
import numpy as np
from matplotlib import colors

In [ ]:
'''Scan with Strand was through terminal - ask how to approach 
(either through terminal on notebook or using gimmemotifs Python API)
Could potentially use snakemake file or bash script to work through this all'''

In [4]:
# Input File
df_scan = pd.read_csv('~/nakahashi_files/Nakahashicore.bed', sep='\t', comment='#', 
                         names=['chrom', 'start', 'end', 'name', 'score', 'strand'])

df_scan = df_scan.drop(columns=['name', 'score'])
df_scan[0:3]

#df_scan_up = bf.read_table('~/N_up.bed', schema='bed9', comment='#').keys()

In [11]:
##Test Case
df1 = pd.DataFrame([
    ['chr1', 1, 5],
    ['chr1', 3, 8],
    ['chr1', 8, 10],
    ['chr1', 12, 14]],
    columns=['chrom', 'start', 'end']
)
display(df1.columns)

df2 = pd.DataFrame([
    ['chr1', 1, 5],
    ['chr1', 3, 8],
    ['chr1', 8, 10],
    ['chr1', 12, 14]],
    columns=['chrom', 'start', 'end']
)
display(df2.columns)

import bioframe as bf

overlapping_intervals = bf.overlap(df1, df2, how = 'inner', suffixes = ('_1', '_2'))
display (overlapping_intervals)

Index(['chrom', 'start', 'end'], dtype='object')

Index(['chrom', 'start', 'end'], dtype='object')

,chrom_1,start_1,end_1,chrom_2,start_2,end_2
0,chr1,1,5,chr1,1,5
1,chr1,1,5,chr1,3,8
2,chr1,3,8,chr1,3,8
3,chr1,8,10,chr1,8,10
4,chr1,12,14,chr1,12,14
5,chr1,3,8,chr1,1,5


In [5]:
#Combining DFs to identify overlaps
overlapping_intervals = bf.overlap(df_Nakahashi, df_scan, how = 'inner', suffixes = ('_1', '_2'))
display (overlapping_intervals)

,chrom_1,start_1,end_1,chrom_2,start_2,end_2,strand_2
0,chr18,3005830,3005969,chr18,3005860,3005864,+
1,chr18,3039452,3039632,chr18,3039475,3039479,+
2,chr18,3280703,3280942,chr18,3280856,3280860,-
3,chr18,3337434,3337836,chr18,3337697,3337701,+
4,chr18,3495516,3495762,chr18,3495573,3495577,-
...,...,...,...,...,...,...,...
38935,chr16,98026545,98026859,chr16,98026597,98026601,-
38936,chr16,98100791,98101039,chr16,98100900,98100904,-
38937,chr16,98158558,98158780,chr16,98158689,98158693,+
38938,chr16,98243951,98244098,chr16,98243964,98243968,-


In [6]:
#Splitting DFs into Pos and Neg Strands

df_pos = overlapping_intervals[overlapping_intervals['strand_2'] == '+']
df_neg = overlapping_intervals[overlapping_intervals['strand_2'] == '-']

## 1 is positive-strand 
## 0 is negative-strand

In [7]:
def create_pos_downstream(df_pos):
    df_downstream_1 = df_pos.copy()
     #display(df_downstream_1)
        
    df_downstream_1['start_1'] = df_downstream_1['start_1'].astype(int)
    df_downstream_1['end_1'] = df_downstream_1['end_1'].astype(int)
    df_downstream_1['start_1'] = df_downstream_1['end_1']
    df_downstream_1['end_1'] = df_downstream_1['start_1'] + 200

    df_downstream_1 = df_downstream_1.drop(columns=['chrom_2', 'start_2','end_2','strand_2'])
    df_downstream_1.columns.values[0] = '#chr'
    df_downstream_1.columns.values[1] = 'start'
    df_downstream_1.columns.values[2] = 'end'
    df_downstream_1.to_csv('downstream_pos.bed', sep ='\t', index = False)
    downstream_pos = df_downstream_1 
    return downstream_pos

,chrom,start,end,strand
0,chr18,3005969,3006169,+
1,chr18,3039632,3039832,+
3,chr18,3337836,3338036,+


In [8]:
def create_pos_upstream(df_pos):
    df_upstream_1 = df_pos.copy()
    #display(df_upstream_1)
    
    df_upstream_1['start_1'] = df_upstream_1['start_1'].astype(int)
    df_upstream_1['end_1'] = df_upstream_1['end_1'].astype(int)
    df_upstream_1['end_1'] = df_upstream_1['start_1']
    df_upstream_1['start_1'] = df_upstream_1['start_1'] - 200
    
    df_upstream_1 = df_upstream_1.drop(columns=['chrom_2', 'start_2','end_2','strand_2'])
    df_upstream_1.columns.values[0] = '#chr'
    df_upstream_1.columns.values[1] = 'start'
    df_upstream_1.columns.values[2] = 'end'
    df_upstream_1.to_csv('upstream_pos.bed', sep ='\t', index = False)
    upstream_pos = df_upstream_1 
    return upstream_pos

In [9]:
def create_neg_upstream(df_neg):
    df_upstream_0 = df_neg.copy()
    #display(df_upstream_0)

    df_upstream_0['start_1'] = df_upstream_0['start_1'].astype(int)
    df_upstream_0['end_1'] = df_upstream_0['end_1'].astype(int)
    df_upstream_0['end_1'] = df_upstream_0['start_1']
    df_upstream_0['start_1'] = df_upstream_0['start_1'] - 200

    df_upstream_0 = df_upstream_0.drop(columns=['chrom_2', 'start_2','end_2','strand_2'])
    df_upstream_0.columns.values[0] = '#chr'
    df_upstream_0.columns.values[1] = 'start'
    df_upstream_0.columns.values[2] = 'end'
    df_upstream_0.to_csv('upstream_neg.bed', sep ='\t', index = False)
    upstream_neg = df_upstream_0 
    return upstream_neg


In [10]:
def create_neg_downstream(df_neg):
    df_downstream_0 = df_neg.copy()
    #display(df_downstream_0)
    
    df_downstream_0['start_1'] = df_downstream_0['start_1'].astype(int)
    df_downstream_0['end_1'] = df_downstream_0['end_1'].astype(int)
    df_downstream_0['start_1'] = df_downstream_0['end_1']
    df_downstream_0['end_1'] = df_downstream_0['start_1'] + 200

    df_downstream_0 = df_downstream_0.drop(columns=['chrom_2', 'start_2','end_2','strand_2'])
    df_downstream_0.columns.values[0] = '#chr'
    df_downstream_0.columns.values[1] = 'start'
    df_downstream_0.columns.values[2] = 'end'
    df_downstream_0.to_csv('downstream_neg.bed', sep ='\t', index = False)
    downstream_neg = df_downstream_0 
    return downstream_neg

In [ ]:
## This code performs de novo motif discovery on the dataframes
'''I have to switch conda environments - there are some dependencies that prevent me 
from having bioframe and gimmemotifs within the same env. '''

from gimmemotifs.denovo import gimme_motifs
from gimmemotifs.scanner import Scanner

peaks = ['upstream_pos.bed', 'downstream_pos.bed', 'upstream_neg.bed', 'upstream_neg.bed']
outdir = ['upstream_pos.bed', 'downstream_pos.bed', 'upstream_neg.bed', 'upstream_neg.bed']
params = {
    "tools": "Homer,BioProspector,MEME",
    "genome": "mm9"
    }

for (x,y) in zip(peaks, outdir): 
    motifs = gimme_motifs(x, y, params=params)

In [78]:
## Visualization of Findings through Heatmap 
## two nice colormaps
cmap = colors.ListedColormap([
  'mistyrose', 
  'darkseagreen',
  'plum',
  'beige'])

cmap_rhee = colors.ListedColormap([
    'red', #a red
    'blue', #c blue
    'yellow', #g yllow
    'green' #t green
])

## dna utilties
def dna_rc(seq):
  return seq.translate(str.maketrans("ATCGatcg","TAGCtagc"))[::-1]

def dna_1hot_index(seq):
  seq_len = len(seq)
  seq = seq.upper()
  # map nt's to a len(seq) of 0,1,2,3
  seq_code = np.zeros(seq_len, dtype='uint8')
  for i in range(seq_len):
    nt = seq[i]
    if nt == 'A':
      seq_code[i] = 0
    elif nt == 'C':
      seq_code[i] = 1
    elif nt == 'G':
      seq_code[i] = 2
    elif nt == 'T':
      seq_code[i] = 3
    else:
      seq_code[i] = random.randint(0,3)         
  return seq_code

import pysam
genome_open = pysam.Fastafile('/project/fudenber_735/genomes/mm10/mm10.fa')


## assuming sites is a dataFrame of motif positions
def heatmap(sites): 
    pad = 100
    sites_dna_num = []
    #sites = sites.astype({'start':int,'end':int'})
    
    
    for i in range(len(sites)):
        if np.mod(i, 100)==0: print(i)
        chrom = sites.loc[[i]][['chrom']]
        start = sites.loc[[i]][['start']].astype(int)
        end = sites.loc[[i]][['end']].astype(int)
        strand = sites.loc[[i]][['strand']]
        start = start - pad
        end = end + pad
        print(type(pad))
        print(type(start))
        print(type(end))
        #chrom, start, end, strand = sites.loc[[i]][['chrom','start','end','strand']]
        ''' Error occuring here with genome_open.fetch - the program wants integers, but I am unable to cast them as integers 
        despite deliberately doing so  (Cast Variable Types) '''
        seq =  genome_open.fetch(chrom, start, end).upper()
        if strand == '-':
            seq = dna_rc(seq)
        sites_dna_num.append(dna_1hot_index(seq))
    sites_dna_num = np.array(sites_dna_num)

    ## optional: cluster by hamming distance between sequences in the 20bp upstream of the motif
    ## and use this to re-order the heatmap of sequences.
    sub_index = (pad-20,pad)
    seq_dist = np.zeros((len(sites),len(sites)))
    for i in range(len(sites)):
        seq_i = sites_dna_num[i][sub_index[0]:sub_index[1]]
        for j in range(len(sites_strong)):
            if i<j:
                seq_j = sites_dna_num[j][sub_index[0]:sub_index[1]]
                seq_dist[i,j] = 1 - scipy.spatial.distance.hamming(seq_i,seq_j)
    seq_dist = seq_dist + seq_dist.T           
    reordering = scipy.cluster.hierarchy.leaves_list(
                scipy.cluster.hierarchy.linkage(seq_dist_strong_tail))

    ## plot the sequences
    plt.figure(figsize=(4,18))
    im = plt.matshow(
        sites_strong_dna[reordering], 
        cmap=cmap_rhee, fignum=False) 
    plt.xlim([70,150]);
    plt.colorbar(im, fraction=0.046, pad=0.04)


display(sites)
#print(bf.is_bedframe(sites))
heatmap(sites)

,chrom,start,end,strand
0,chr18,3005969,3006169,+
1,chr18,3039632,3039832,+
3,chr18,3337836,3338036,+
5,chr18,3497634,3497834,+
6,chr18,3508038,3508238,+
...,...,...,...,...
38932,chr16,97790546,97790746,+
38933,chr16,97815065,97815265,+
38934,chr16,97889463,97889663,+
38937,chr16,98158780,98158980,+


0
<class 'int'>
<class 'pandas.core.frame.DataFrame'>
<class 'pandas.core.frame.DataFrame'>


TypeError: an integer is required

In [ ]:
## Visualization of Findings through Heatmap for all Strands and Upstream and Downstream
df_list = ['upstream_pos.bed', 'downstream_pos.bed', 'upstream_neg.bed', 'upstream_neg.bed']
for df in df_list:
    sites = df
    heatmap(sites) 